In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
from sci378 import *

Version:  0.0.7


In [3]:
from lmfit import *

## Crichton problem

- need temperature vs time data for all of the station (csv files in one folder)
- need rural/urban info for each station
- location (latitude, logitude) for each station


- for each file do ....
    - load data
    - fit to a line
    - get the slope of the line
    - store those values


- average of all the slopes?  (X)
- count all positives/negatives for rural, urban -- fraction of positive for rural vs urban
- average urban average rural, look statistical test on the average

- go through all the urban stations
    - find the closest rural station
    - I already have the slopes - rural slope value, and an urban slope value
    
- plot the difference (histogram)
- plot rural slope (y-axis) vs urban slope (x-axis)


In [4]:
from glob import glob

In [5]:
time_series_folder='/Users/bblais/tex/bryant/spring_2021/computer_programming_378/Crichton Data/time series'
fnames=glob(time_series_folder+'/*.csv')
fnames=fnames[:10]

In [6]:
len(fnames)

10

In [7]:
fnames

['/Users/bblais/tex/bryant/spring_2021/computer_programming_378/Crichton Data/time series/CARACAS-LA CARLOTA.csv',
 '/Users/bblais/tex/bryant/spring_2021/computer_programming_378/Crichton Data/time series/TAREE.csv',
 '/Users/bblais/tex/bryant/spring_2021/computer_programming_378/Crichton Data/time series/YAXIAN.csv',
 '/Users/bblais/tex/bryant/spring_2021/computer_programming_378/Crichton Data/time series/MOANDA.csv',
 '/Users/bblais/tex/bryant/spring_2021/computer_programming_378/Crichton Data/time series/MINDEN.csv',
 '/Users/bblais/tex/bryant/spring_2021/computer_programming_378/Crichton Data/time series/NORFOLK INTL AP.csv',
 '/Users/bblais/tex/bryant/spring_2021/computer_programming_378/Crichton Data/time series/BRISBANE REGIONAL OFFI.csv',
 '/Users/bblais/tex/bryant/spring_2021/computer_programming_378/Crichton Data/time series/CHARLOTTESVILLE 2W.csv',
 '/Users/bblais/tex/bryant/spring_2021/computer_programming_378/Crichton Data/time series/CLEVELAND-CLE.csv',
 '/Users/bblais/te

In [8]:
station_info=pd.read_excel('data/station_info.xlsx')
station_info.head()

,ID,Station,Latitude,Longitude,Brightness,Elevation
0,ACW00011604,SAVE,57.7667,11.8667,45.0,18.0
1,AE000041196,SHARJAH_INTER_AIRP,25.3330,55.5170,39.0,34.0
2,AEM00041184,RAS_AL_KHAIMAH_INTE,25.6170,55.9330,17.0,31.0
3,AEM00041194,DUBAI_INTL,25.2550,55.3640,104.0,10.4
4,AEM00041216,ABU_DHABI_BATEEN_AIR,24.4300,54.4700,114.0,3.0


In [31]:
def get_slope(fname,plotit=False):
    data=pd.read_csv(fname)
    data=data.dropna()
    x_data=data['Year']
    y_data=data['Temperature']
    model=models.LinearModel()
    result=model.fit(y_data,x=x_data)
    
    if plotit:
        plot(x_data,y_data,'o')

        x_fake=linspace(1960,2020,100)
        y_fake=result.eval(x=x_fake)
        plot(x_fake,y_fake,'-')        
    
    return result.params['slope'].value

def station_name_from_filename(filename):
    import os
    parts=os.path.split(filename)
    base,ext=os.path.splitext(parts[-1])
    return base

def first_row(station_name):
    if '.csv' in station_name:
        station_name=station_name_from_filename(station_name)
    
    if not any(station_info['Station']==station_name):
        return None
    
    return station_info[station_info['Station']==station_name].iloc[0]    

In [32]:
station_name='CARACAS-LA CARLOTA'
any(station_info['Station']==station_name)

False

In [33]:
fnames=glob(time_series_folder+'/*.csv')
station_names_with_data=[station_name_from_filename(filename) for filename in fnames]
#fnames=fnames[:10]

In [36]:
S=Storage()

count=0
for filename in fnames:
    data=Struct()
    data.slope=get_slope(filename)
    data.station=station_name_from_filename(filename)
    
    row=first_row(data.station)
    if row is None:
        count+=1
        continue
    if row.Brightness>=10:
        data.urban=True
    else:
        data.urban=False
    
    S+=data,
    
print("Could not find info for ",count)

Could not find info for  3588


In [28]:
data.station

'CARACAS-LA CARLOTA'

In [86]:
first_row(station_name_from_filename(fnames[3]))

ID            GB000004565
Station            MOANDA
Latitude            -1.53
Longitude           13.27
Brightness            0.0
Elevation           572.0
Name: 7521, dtype: object

In [87]:
first_row(fnames[3])

ID            GB000004565
Station            MOANDA
Latitude            -1.53
Longitude           13.27
Brightness            0.0
Elevation           572.0
Name: 7521, dtype: object

In [84]:
fnames[3]

'/Users/bblais/Desktop/time series/MOANDA.csv'